# 뉴스-종목 매핑 모델링
1. 과거 뉴스의 종목 라벨링 NER
2. 뉴스 본문 요약 후 임베딩해서 피쳐 787개 → PCA 150개 줄이기
3. 멀티라벨 분류 모델링

In [67]:
!pip install transformers
!pip install torch
!pip install pandas

In [68]:
# 모델 및 파이프라인 불러오기
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# 한국어 NER 모델
model_name = "KPF/KPF-BERT-NER"

# 파이프라인 구성
ner_pipeline = pipeline(
    task="ner",
    model=model_name,
    tokenizer=model_name,
    aggregation_strategy="simple",
    framework="pt"
)

Device set to use mps:0


In [69]:
# 종목명 사전 불러오기
import pandas as pd

company_df = pd.read_csv("../fastapi/datas/KRX_기업명_20250521.csv")
company_names = set(company_df["종목명"].dropna().unique())

In [87]:
company_names

{'AJ네트웍스',
 'AK홀딩스',
 'BGF',
 'BGF리테일',
 'BNK금융지주',
 'BYC',
 'CJ',
 'CJ CGV',
 'CJ대한통운',
 'CJ씨푸드',
 'CJ제일제당',
 'CR홀딩스',
 'CS홀딩스',
 'DB',
 'DB손해보험',
 'DB증권',
 'DB하이텍',
 'DH오토넥스',
 'DI동일',
 'DKME',
 'DL',
 'DL이앤씨',
 'DN오토모티브',
 'DRB동일',
 'DSR',
 'DSR제강',
 'DS단석',
 'DYP',
 'E1',
 'ESR켄달스퀘어리츠',
 'F&F',
 'F&F홀딩스',
 'GKL',
 'GS',
 'GS건설',
 'GS글로벌',
 'GS리테일',
 'GS피앤엘',
 'HDC',
 'HDC랩스',
 'HDC현대EP',
 'HDC현대산업개발',
 'HD한국조선해양',
 'HD현대',
 'HD현대건설기계',
 'HD현대마린솔루션',
 'HD현대마린엔진',
 'HD현대미포',
 'HD현대에너지솔루션',
 'HD현대인프라코어',
 'HD현대일렉트릭',
 'HD현대중공업',
 'HJ중공업',
 'HL D&I',
 'HLB글로벌',
 'HL만도',
 'HL홀딩스',
 'HMM',
 'HS애드',
 'HS화성',
 'HS효성',
 'HS효성첨단소재',
 'IHQ',
 'INVENI',
 'JB금융지주',
 'JW생명과학',
 'JW중외제약',
 'JW홀딩스',
 'KB금융',
 'KB발해인프라',
 'KB스타리츠',
 'KCC',
 'KCC글라스',
 'KCTC',
 'KC그린홀딩스',
 'KC코트렐',
 'KEC',
 'KG모빌리티',
 'KG스틸',
 'KG케미칼',
 'KH 필룩스',
 'KISCO홀딩스',
 'KPX케미칼',
 'KPX홀딩스',
 'KR모터스',
 'KSS해운',
 'KT',
 'KT&G',
 'KTcs',
 'KTis',
 'LF',
 'LG',
 'LG디스플레이',
 'LG생활건강',
 'LG씨엔에스',
 'LG에너지솔루션',
 'LG유플러스',
 'LG이노텍',
 

In [70]:
# 뉴스 데이터 불러오기
news_df = pd.read_csv("../fastapi/datas/mknews_with_id_embedding.csv")

# 제목 + 본문 결합
news_df["text_combined"] = (
    news_df["title"].fillna("") + " " + news_df["content"].fillna("")
).str.replace(r"\s+", " ", regex=True).str.strip()

In [89]:
news_df['text_combined'][0]

'배터리도 증시 대피처 아니었다…한달 동안 -23% 빠진 LG엔솔 경기침체로 전기차 수요 부진 우려가 불거지면서 코스피 시총 2위 LG에너지솔루션의 주가가 최근 한달여 사이에 23% 가량 하락한 것으로 나타났다. [제공 : LG에너지솔루션] 2차전지 대장주 LG에너지솔루션이 ‘믿을 건 2차전지 밖에 없다’던 투자자들의 발등을 찍고 있다. 지난 10~11월 코스피 반등장을 주도하면서 주목을 받았지만 한달여만에 20% 넘는 주가 하락을 보이고 있기 때문이다. 19일 증권가에 따르면 LG에너지솔루션은 전일대비 3500원(0.72%) 내린 48만2000원에 마감했다. LG에너지솔루션의 주가는 이번달에만 17.89%나 하락했다. 전고점인 지난달 11일 62만9000원 대비로는 23.37%나 떨어졌다. LG에너지솔루션은 이달 들어 13거래일 가운데 이날을 포함해 11거래일 동안 하락하는 등 극도의 주가 부진을 나타내고 있다. 올 하반기 LG에너지솔루션이 코스피에 비해 견조한 흐름을 보인 것과는 대비되는 모습이다. 지난 10월과 11월 1개월 동안 LG에너지솔루션 주가는 각각 23.80%, 11.17%나 상승했다. 같은 기간 코스피 상승률 6.41%, 7.80%를 크게 앞지르는 숫자다. 코스피가 바닥을 찍었던 지난 9월에도 코스피가 월간 기준으로 12.81%나 하락했지만 LG에너지솔루션의 하락폭은 7.78%에 그쳤다. 수급면에서는 외국인의 매수세가 끊긴 점이 눈에 띈다. 외국인 투자자는 지난 10월과 11월 각각 4399억원, 8261억원을 순매수했다. 10월에는 외국인 순매수 2위, 11월에는 1위가 LG에너지솔루션이었다. 하지만 이달 들어서는 223억원 순매도로 돌아섰다. LG에너지솔루션의 최근 1개월간 주가 추이. [출처 : 구글 파이낸스] LG에너지솔루션의 주가가 갑자기 우하향으로 방향을 튼 것은 전기차 시장에 대한 비관론이 고개를 들고 있기 때문이다. 경기가 침체되면 전기차 판매도 감소할 것이란 우려다. 경기 침체 우려가 점점 더 커지는 와중에 배터리 관련주 주가

In [ ]:
# 뉴스 텍스트를 500자씩 나누기
def split_text(text, max_length=500):
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

# 종목명 추출 함수
def extract_stock_labels(text):
    try:
        chunks = split_text(text, max_length=500)
        matched_names = set()
        for chunk in chunks:
            entities = ner_pipeline(chunk)
            org_names = [e['word'] for e in entities if e['entity_group'] == 'LABEL_231']
            matched_names.update(set(org_names) & company_names)
        return list(matched_names)
    except Exception as e:
        print("NER 오류 발생:", e)
        return []

In [92]:
chunk = split_text(news_df['text_combined'][0], max_length=500)
chunk

entities_test = ner_pipeline(chunk)

In [97]:
entities_test

[[{'entity_group': 'LABEL_130',
   'score': 0.99976355,
   'word': '배터리',
   'start': 0,
   'end': 3},
  {'entity_group': 'LABEL_299',
   'score': 0.99998957,
   'word': '##도 증시 대피처 아니었다 …',
   'start': 3,
   'end': 17},
  {'entity_group': 'LABEL_44',
   'score': 0.9987907,
   'word': '한',
   'start': 17,
   'end': 18},
  {'entity_group': 'LABEL_194',
   'score': 0.9999759,
   'word': '##달 동안',
   'start': 18,
   'end': 22},
  {'entity_group': 'LABEL_114',
   'score': 0.99988484,
   'word': '-',
   'start': 23,
   'end': 24},
  {'entity_group': 'LABEL_264',
   'score': 0.9999687,
   'word': '23 %',
   'start': 24,
   'end': 27},
  {'entity_group': 'LABEL_299',
   'score': 0.99999976,
   'word': '빠진',
   'start': 28,
   'end': 30},
  {'entity_group': 'LABEL_81',
   'score': 0.9997403,
   'word': 'LG',
   'start': 31,
   'end': 33},
  {'entity_group': 'LABEL_231',
   'score': 0.9999861,
   'word': '##엔솔',
   'start': 33,
   'end': 35},
  {'entity_group': 'LABEL_299',
   'score': 0.999745

In [99]:
org_names = [e['word'] for e in entities_test[0] if e['entity_group'] == 'ORG']
org_names

[]

In [80]:
from tqdm import tqdm
import time

results = []
start_time = time.time()

for idx, text in tqdm(enumerate(news_df["text_combined"]), total=len(news_df), desc="NER 처리중"):
    try:
        matched = extract_stock_labels(text)
    except Exception as e:
        print(f"NER 오류 발생 (인덱스 {idx}): {e}")
        matched = []
    results.append(matched)

    elapsed = time.time() - start_time
    avg_time = elapsed / (idx + 1)
    remaining = avg_time * (len(news_df) - idx - 1)
    tqdm.write(f"진행률: {idx+1}/{len(news_df)} - 예상 남은 시간: {remaining:.1f}초")

news_df["labels_bert"] = results

NER 처리중:   0%|          | 2/9005 [00:00<24:30,  6.12it/s]   

진행률: 1/9005 - 예상 남은 시간: 2209.6초
진행률: 2/9005 - 예상 남은 시간: 1590.6초


NER 처리중:   0%|          | 5/9005 [00:00<17:18,  8.66it/s]   

진행률: 3/9005 - 예상 남은 시간: 1523.0초
진행률: 4/9005 - 예상 남은 시간: 1231.5초
진행률: 5/9005 - 예상 남은 시간: 1208.6초


NER 처리중:   0%|          | 7/9005 [00:00<14:49, 10.12it/s]   

...

진행률: 9002/9005 - 예상 남은 시간: 0.4초
진행률: 9003/9005 - 예상 남은 시간: 0.3초


NER 처리중: 100%|█████████▉| 9004/9005 [21:50<00:00,  5.78it/s]   

진행률: 9004/9005 - 예상 남은 시간: 0.1초


NER 처리중: 100%|██████████| 9005/9005 [21:50<00:00,  6.87it/s]   

진행률: 9005/9005 - 예상 남은 시간: 0.0초


In [86]:
news_df[["news_id", "title", "labels_bert"]][100:200]

,news_id,title,labels_bert
100,20230118_0008,"“국내증시에 환멸이”...동학개미, 이 종목 제일 많이 팔았다",[]
101,20230118_0009,"“5천 넣으면 1억 꽂힌다”…‘투자자 울린’ 주식 리딩방, 과태료 추진",[]
102,20230118_0010,2개월만에 추천 투자전략 바꾼 삼성증권…“지금은 주식 사야할 때”,[]
103,20230118_0011,"토종 PEF 센트로이드가 품은 테일러메이드 , ‘골프황제’ 타이거 우즈도 찜한 美 ...",[]
104,20230118_0012,‘패션+K팝’ 꿀조합에 베팅…주식 초고수가 찍은 이 종목,[]
...,...,...,...
195,20230127_0004,금융당국 한마디에...잘 나가던 금융지주株 급브레이크,[]
196,20230127_0005,"바이오니아, 결핵·다제내성 결핵균 검사키트 식약처 수출허가 획득",[]
197,20230127_0006,김소영 금융위 부위원장 “ESG 공시제도 구체화해야”,[]
198,20230127_0007,"LG엔솔, 4조 규모 보호예수 해제 임박...투자자 ‘긴장’",[]
